# Section 504.3 - Height in feet
## Building Code
The maximum height, in feet, of a building shall not exceed the limits specified in Table 504.3. 

**Exception**: Towers, spires, steeples and other roof structures shall be constructed of materials consistent with the required type of construction of the building except where other construction is permitted by Section 1510.2.5. Such structures shall not be used for habitation or storage. The structures shall be unlimited in height where of noncombustible materials and shall not extend more than 20 feet (6096 mm) above the allowable building height where of combustible materials (see Chapter 15 for additional requirements). 

### TABLE 504.3a ALLOWABLE BUILDING HEIGHT IN FEET ABOVE GRADE PLANE  

In [11]:
import json

# Load the table from the JSON file
with open('table_504_3.json', 'r') as f:
    table_504_3 = json.load(f)


In [3]:
import pandas as pd
# Table 504.3 Display
from IPython.display import display, HTML

rows = []

for occupancy_group, systems in table_504_3.items():
    for sprinkler_system, types in systems.items():
        for construction_type, subtypes in types.items():
            for subtype, height_limit in subtypes.items():
                rows.append({
                    "Occupancy Group": occupancy_group,
                    "Sprinkler System": sprinkler_system,
                    "Construction Type": construction_type,
                    "Subtype": subtype,
                    "Height Limit": height_limit
                })

df = pd.DataFrame(rows)

# Display the DataFrame as a scrollable table
def render_scrollable_table(dataframe, max_height=300):
    display(HTML(f"""
    <div style="max-height: {max_height}px; overflow-y: scroll;">
        {dataframe.to_html(index=False)}
    </div>
    """))

render_scrollable_table(df)

Occupancy Group,Sprinkler System,Construction Type,Subtype,Height Limit
"A, A-2, A-3, B, E, F, M, S, U",NS,Type I,A,UL
"A, A-2, A-3, B, E, F, M, S, U",NS,Type I,B,160
"A, A-2, A-3, B, E, F, M, S, U",NS,Type II,A,65
"A, A-2, A-3, B, E, F, M, S, U",NS,Type II,B,55
"A, A-2, A-3, B, E, F, M, S, U",NS,Type III,A,65
"A, A-2, A-3, B, E, F, M, S, U",NS,Type III,B,55
"A, A-2, A-3, B, E, F, M, S, U",NS,Type IV,HT,65
"A, A-2, A-3, B, E, F, M, S, U",NS,Type V,A,50
"A, A-2, A-3, B, E, F, M, S, U",NS,Type V,B,40
"A, A-2, A-3, B, E, F, M, S, U",S,Type I,A,UL


For SI: 1 foot = 304.8 mm.

**Note:**

- UL = Unlimited
- NS = Buildings not equipped throughout with an automatic sprinkler system
- S = Buildings equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.1
- S13R = Buildings equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.2

a. See Chapters 4 and 5 for specific exceptions to the allowable height in this chapter.

b. See Section 903.2 for the minimum thresholds for protection by an automatic sprinkler system for specific occupancies.

c. New Group H occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.5.

d. The NS value is only for use in evaluation of existing building height in accordance with the International Existing Building Code.

e. New Group I-1 and I-3 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6. For new Group I-1 occupancies Condition 1, see Exception 1 of Section 903.2.6.

f. New and existing Group I-2 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6 and Section 1103.5 of the International Fire Code.

g. For new Group I-4 occupancies, see Exceptions 2 and 3 of Section 903.2.6.

h. New Group R occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.8.

In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Set up the SPARQL endpoint
sparql = SPARQLWrapper("http://localhost:3030/SmartReviewHotel/sparql")

In [10]:
# Getting Construction Type, Occupancy Group, & Sprinkler Type for the building
with open("..\..\..\SPARQL\storeyRule.sparql", "r") as file:
    sparql_query = file.read()

sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Process the results
for result in results["results"]["bindings"]:
    construction_type = result["constructionType"]["value"]
    occupancy_group = result["occupancyGroup"]["value"]
    sprinkler_type = result["sprinklerType"]["value"] if "sprinklerType" in result else "none"
    storey_num = result["storeyNumValue"]["value"]

    # Splitting the construction type and subtype
    if "-" in construction_type:
        main_construction_type, subtype = construction_type.rsplit("-", 1)
    else:
        main_construction_type = construction_type
        subtype = ""

    # Determine the sprinkler system type
    sprinkler_system = "S" if sprinkler_type != "none" else "NS"

    #print(main_construction_type, subtype, occupancy_group, sprinkler_system)

In [8]:
import sys
import os

# Add the path to the Rule Notebooks folder to the system path
sys.path.append(os.path.abspath('../..'))

# Import the find_value function from table_lookup.py
from ruleFunctions import find_value, check_value

In [9]:
building_height = 57.33 #hardcoded value for now
allowed_building_height = find_value(table_504_3, occupancy_group, sprinkler_system, main_construction_type, subtype)
building_height_result = check_value(building_height, allowed_building_height)

print(f"This section is considered: {building_height_result}")
if building_height_result == "Passed":
    passed5043 = "does not"
else: 
    passed5043 = "does"

print(f"\nThe building height of {building_height} ft {passed5043} exceed the limit of {allowed_building_height} ft as specified in Table 504.3 for:\n" \
                             f"• Occupancy - {occupancy_group},\n" \
                             f"• Construction Type - {main_construction_type},\n" \
                             f"• Subtype - {subtype},\n" \
                             f"• Sprinkler System - {sprinkler_type}.")

This section is considered: Passed

The building height of 57.33 ft does not exceed the limit of 70 ft as specified in Table 504.3 for:
• Occupancy - R-1,
• Construction Type - Type V,
• Subtype - A,
• Sprinkler System - 13.


The Exception allowing for increased heights for towers, spires, steeples, and other roof structures has not been implemented. Please verify this exception manually.